In [30]:
import numpy as np
import pandas as pd
import h5py
from astropy.io import fits
import os
import matplotlib.pyplot as plt
import pickle
import pandas as pd
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy.wcs.utils import skycoord_to_pixel
import astropy.units as u
import numpy as np

In [31]:
file_path = "/Users/yi/Documents/CMB_SZ/Herschel catalog/mask/HATLAS_GAMA9_DR1_MASK500.FITS"
hdul = fits.open(file_path)

# Extract WCS information from the header
wcs = WCS(hdul[0].header)

# Print WCS 
print(wcs)


tan:{cdelt:[-0.003333,0.003333],crval:[136.6,0.4045],crpix:[1684.00,871.00]}


In [32]:

def mask_catalog(mask_path, catalog_df): 

    # Load the Herschel mask and WCS
    mask_file_path = mask_path
    hdul = fits.open(mask_file_path)
    mask_data = hdul[0].data  # 2D numpy array of the mask
    wcs = WCS(hdul[0].header)  # WCS object for pixel-to-world mapping
    hdul.close()

    # Load the catalog
    #catalog = pd.read_csv("catalog.csv")  # file check 
    if "ra" not in catalog_df.columns or "dec" not in catalog_df.columns:
        raise ValueError("The catalog must contain 'ra' and 'dec' columns.")

    ra, dec = catalog_df['ra'].values, catalog_df['dec'].values  # Replace 'ra' and 'dec' with actual column names

    # Convert RA/Dec to pixel coordinates
    sky_coords = SkyCoord(ra * u.deg, dec * u.deg)
    x_pix, y_pix = skycoord_to_pixel(sky_coords, wcs)

    # Round pixel coordinates to integers
    x_pix = np.round(x_pix).astype(int)
    y_pix = np.round(y_pix).astype(int)

    # Get mask dimensions
    mask_shape = mask_data.shape

    # Ensure pixel coordinates are within the bounds of the mask
    valid_pixels = (x_pix >= 0) & (x_pix < mask_shape[1]) & (y_pix >= 0) & (y_pix < mask_shape[0])

    # Only index mask_data with valid pixel indices
    y_valid = y_pix[valid_pixels]
    x_valid = x_pix[valid_pixels]
    in_mask = np.zeros(len(catalog), dtype=bool)  # Initialize mask

    in_mask[valid_pixels] = mask_data[y_valid, x_valid] > 0  # Apply mask check

    # Apply the mask to the catalog
    filtered_catalog = catalog_df[in_mask]

    # Save the filtered catalog
    #filtered_catalog.to_csv("filtered_catalog.csv", index=False)

    print(f"Filtered catalog saved with {len(filtered_catalog)} sources.")

    return filtered_catalog



In [38]:
mask_GAMA9 = "/Users/yi/Documents/CMB_SZ/Herschel catalog/mask/HATLAS_GAMA9_DR1_MASK500.FITS"
catalog = pd.read_csv("/Users/yi/Documents/CMB_SZ/ACTxDESI/V20_DR15_Catalog_v3.csv", delimiter=',', skiprows = 17)  # please manually count the rows of flags 
gama9_catalog = mask_catalog(mask_GAMA9, catalog)

Filtered catalog saved with 4766 sources.


/var/folders/t2/dlv66srx743g81cmz1pqw0200000gn/T/ipykernel_11607/1760335801.py:22: RuntimeWarning: invalid value encountered in cast
  x_pix = np.round(x_pix).astype(int)
/var/folders/t2/dlv66srx743g81cmz1pqw0200000gn/T/ipykernel_11607/1760335801.py:23: RuntimeWarning: invalid value encountered in cast
  y_pix = np.round(y_pix).astype(int)


In [37]:
mask_GAMA12 = "/Users/yi/Documents/CMB_SZ/Herschel catalog/mask/HATLAS_GAMA12_DR1_MASK500.FITS"
gama12_catalog = mask_catalog(mask_GAMA12, catalog)

Filtered catalog saved with 4400 sources.


/var/folders/t2/dlv66srx743g81cmz1pqw0200000gn/T/ipykernel_11607/1760335801.py:22: RuntimeWarning: invalid value encountered in cast
  x_pix = np.round(x_pix).astype(int)
/var/folders/t2/dlv66srx743g81cmz1pqw0200000gn/T/ipykernel_11607/1760335801.py:23: RuntimeWarning: invalid value encountered in cast
  y_pix = np.round(y_pix).astype(int)


In [39]:
mask_GAMA15 = "/Users/yi/Documents/CMB_SZ/Herschel catalog/mask/HATLAS_GAMA15_DR1_MASK500.FITS"
gama15_catalog = mask_catalog(mask_GAMA15, catalog)

Filtered catalog saved with 6152 sources.


/var/folders/t2/dlv66srx743g81cmz1pqw0200000gn/T/ipykernel_11607/1760335801.py:22: RuntimeWarning: invalid value encountered in cast
  x_pix = np.round(x_pix).astype(int)
/var/folders/t2/dlv66srx743g81cmz1pqw0200000gn/T/ipykernel_11607/1760335801.py:23: RuntimeWarning: invalid value encountered in cast
  y_pix = np.round(y_pix).astype(int)


In [42]:
final_catalog = pd.concat([gama9_catalog, gama12_catalog,gama15_catalog])

In [43]:
final_catalog

,ra,dec,lum,z,stellarmasses,halomasses,cModelMag_u,cModelMag_g,cModelMag_r,cModelMag_i,...,extinction_z,bestObjID,PS15mJy_cut,PS100mJy_cut,divcut,divcuts18,galcut,incompsepregion,S18coadd,S16ILC
9998,138.56262,-1.868027,5.859838e+10,0.623679,1.757951e+11,9.226897e+12,21.35510,21.70115,21.30538,19.66484,...,0.048826,1.237655e+18,1.0,1.0,2.0,2.0,2.0,0.0,1.0,0.0
9999,138.69037,-1.762424,6.768371e+10,0.485055,2.030511e+11,1.220077e+13,21.74011,22.05253,20.12235,19.24000,...,0.045395,1.237655e+18,1.0,1.0,2.0,2.0,2.0,0.0,1.0,0.0
10000,138.71900,-1.783417,7.008155e+10,0.485812,2.102447e+11,1.306843e+13,22.84764,21.96158,19.96437,18.97590,...,0.045222,1.237655e+18,1.0,1.0,2.0,2.0,2.0,0.0,1.0,0.0
10002,138.76868,-1.790508,2.440482e+10,0.343469,7.321447e+10,2.104770e+12,19.23161,21.12523,19.96945,19.32702,...,0.045889,1.237655e+18,1.0,1.0,2.0,2.0,2.0,0.0,1.0,0.0
10003,138.83689,-1.779718,2.878238e+10,0.490318,8.634713e+10,2.688421e+12,24.60486,22.75964,21.05743,19.91588,...,0.048922,1.237655e+18,1.0,1.0,2.0,2.0,2.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474948,212.22552,-1.290228,1.303118e+11,0.638569,3.909354e+11,4.762477e+13,26.04254,23.03140,20.56776,19.60062,...,0.063538,1.237675e+18,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0
474949,212.39543,-1.358334,6.556186e+10,0.488307,1.966856e+11,1.146216e+13,20.87917,21.89149,20.49156,19.43719,...,0.066817,1.237675e+18,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0
474950,212.30255,-1.345069,5.478938e+10,0.458621,1.643681e+11,8.123726e+12,23.21077,21.62184,20.02060,19.20633,...,0.064682,1.237675e+18,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0
474951,212.48387,-1.485007,7.270276e+10,0.638835,2.181083e+11,1.405747e+13,25.48514,23.77435,21.19671,20.28955,...,0.076326,1.237675e+18,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0


In [44]:
final_catalog.to_csv("V21_GAMA_overlap_catalog.csv", index=False)

In [28]:
sus_catalog = pd.read_csv("filtered_catalog.csv", delimiter=',')  # please manually count the rows of flags 
